데이터 셋 다운로드

In [1]:
import os
import tarfile
import urllib

down_root = 'https://raw.githubusercontent.com/ageron/handson-ml2/master/'
housing_path = os.path.join('datasets', 'housing')
housing_url = down_root + 'datasets/housing/housing.tgz'

def fetch_housing_data(housing_url = housing_url, housing_path = housing_path) : 
    os.makedirs(housing_path, exist_ok = True)
    tgz_path = os.path.join(housing_path, 'housing.tgz')
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path = os.path.join('data'))
    housing_tgz.close()


In [2]:
fetch_housing_data()

데이터 셋 불러오기

In [3]:
import pandas as pd

def load_data(data_path, data_name) : 
    csv_path = os.path.join(data_path, data_name)
    return pd.read_csv(csv_path)

data_path = os.path.join('data')
data_name = 'housing.csv'

housing = load_data(data_path = data_path, data_name = data_name)

In [4]:
housing['ocean_proximity'].value_counts()

<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: ocean_proximity, dtype: int64

파이프라인 구성

In [12]:
from sklearn import pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin

# 열 인덱스
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True): # *args 또는 **kargs 없음
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # 아무것도 하지 않습니다
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

In [11]:
num_pipeline = pipeline.Pipeline([
    ('imputer', SimpleImputer(strategy = 'median')),
    ('attrib_adder', CombinedAttributesAdder()),
    ('std_scaler', pp.StandardScaler()),
])

housing_num_tr = num_pipeline.fit_transform(housing)

ValueError: Cannot use median strategy with non-numeric data:
could not convert string to float: 'NEAR BAY'